In [2]:
import math
import numpy as np
import gym
from gym import spaces

In [25]:
class SingleLinkManipulatorEnv(gym.Env):
    metadata = {
        "render_modes": ["human"],
    }

    def __init__(self):
        self.min_action = -10.0
        self.max_action = 10.0
        self.min_x1 = 0.0
        self.max_x1 = 3.0
        self.min_x2 = 0.0
        self.max_x2 = 1.5
        self.goal_x1 = (math.pi/6.0)
        self.goal_x2 = 0.0

        self.low_state = np.array([self.min_x1, self.min_x2], dtype=np.float32)
        self.high_state = np.array(
            [self.max_x1, self.max_x2], dtype=np.float32)

        self.action_space = spaces.Box(
            low=self.min_action, high=self.max_action, shape=(1,), dtype=np.float32
        )
        self.observation_space = spaces.Box(
            low=self.low_state, high=self.high_state, dtype=np.float32
        )

    def step(self, action: np.ndarray):
        J = 1.625103
        m = 0.506
        M0 = 0.434
        L0 = 0.305
        R0 = 0.023
        B0 = 16.25163
        L = 0.0250103
        R = 5.0
        Kt = 0.90
        Kb = 0.90
        g = 9.8
        M = J + m*L0*L0/3.0 + M0*L0*L0 + 2*M0*R0*R0/5/Kt
        N = m*L0*g/2.0 + M0*L0*g/Kt
        B = B0/Kt

        x1 = self.state[0]
        x2 = self.state[1]
        a = min(max(action[0], self.min_action), self.max_action)

        x3 = (a-Kb*x2)/R
        x1dot = x2
        x2dot = -(N/M)*math.sin(x1) - (B/M)*x2 + (x3/M)
        x1p = x1 + 0.1*x1dot
        x2p = x2 + 0.1*x2dot

        if x1p > self.max_x1:
            x1p = self.max_x1
        if x1p < self.min_x1:
            x1p = self.min_x1
        if x2p > self.max_x2:
            x2p = self.max_x2
        if x2p < self.min_x2:
            x2p = self.min_x2

        terminated = False
        reward = -abs(x1p - self.goal_x1) - abs(x2p-self.goal_x2)
        
        self.state = np.array([x1p, x2p], dtype=np.float32)

        return self.state, reward, terminated, {}

    def reset(self):
        self.state = self.observation_space.sample()
        return np.array(self.state, dtype=np.float32)


In [26]:
from stable_baselines3 import DDPG
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
import matplotlib.pyplot as plt


In [27]:
env = SingleLinkManipulatorEnv()
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(
    n_actions), sigma=0.1 * np.ones(n_actions))
# check_env(env)
model = DDPG('MlpPolicy', env, action_noise=action_noise, verbose=1)
model.learn(100)
metrics = evaluate_policy(model, model.get_env(
), n_eval_episodes=100, return_episode_rewards=True)
plt.plot(metrics[0])


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
